Laying out your report to make it more readable and understandable

# Layout, Pages, and Selects

## Overview

Although you are you probably analyzing data in a list of sequential steps, it may not the best way to present results -- especially if you are sharing with people who are used to a traditional dashboard or BI tool.

Datapane's report library provides components which allow for more flexible grid-style layouts, multiple pages, tabs, and selects, which allows you to create powerful custom interfaces without knowledge of HTML or CSS.

## Grid layouts

If you pass a list of blocks (such as `Plot` and `Table`) to a Report, they are -- by default -- laid out in a single column with a row per block. If you would like to customize the rows and columns, Datapane provides a `Group` component which takes a list of blocks and a number of columns and lays them out in a grid.

If we take the example in the earlier tutorial, but want to lay the plot and dataset side-by-side, we can use specify this using `Group` and specifying the number of columns.

In [7]:
import pandas as pd
import altair as alt
import datapane as dp
alt.data_transformers.disable_max_rows()

dataset = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
df = (
    dataset.groupby(["continent", "date"])["new_cases_smoothed_per_million"]
    .mean()
    .reset_index()
)

plot = (
    alt.Chart(df)
    .mark_area(opacity=0.4, stroke="black")
    .encode(
        x="date:T",
        y=alt.Y("new_cases_smoothed_per_million:Q", stack=None),
        color=alt.Color("continent:N", scale=alt.Scale(scheme="set1")),
        tooltip="continent:N",
    )
    .interactive()
    .properties(width="container")
)

report = dp.Report(dp.Group(dp.Plot(plot), dp.DataTable(df), columns=2))
report = report.save(path="grid-layout.html")

Report saved to ./grid-layout.html

In [10]:
from dpdocsutils import previews
previews.embed_local_report('/tutorials/grid-layout.html', width="100%", height=580)

If you're generating your plots programmatically or have a lot of plots, you can pass them into the Group block as a list, using the `blocks` parameter. We can rewrite the previous example as follows :&#x20;

In [11]:
import pandas as pd
import altair as alt
import datapane as dp

dataset = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
df = (
    dataset.groupby(["continent", "date"])["new_cases_smoothed_per_million"]
    .mean()
    .reset_index()
)

plot = (
    alt.Chart(df)
    .mark_area(opacity=0.4, stroke="black")
    .encode(
        x="date:T",
        y=alt.Y("new_cases_smoothed_per_million:Q", stack=None),
        color=alt.Color("continent:N", scale=alt.Scale(scheme="set1")),
        tooltip="continent:N",
    )
    .interactive()
    .properties(width="container")
)

# You could also generate these in a loop/function
my_plots = [dp.Plot(plot), dp.DataTable(df)]

report = dp.Report(dp.Group(blocks=my_plots, columns=2))

report.save(path="group-list.html")

Report saved to ./group-list.html

In [12]:
from dpdocsutils import previews
previews.embed_local_report('/tutorials/group-list.html', width="100%", height=580)

!!! info
    
    As `Group`components are components themselves, they are composable, and you can create more custom layers of nested blocks, for instance nesting 2 rows in the left column of a 2 column layout

### Customizing width

To customize the width of your report, you can set the [report type](./#report-types).

## Pages&#x20;

Reports on Datapane can have multiple pages, which are presented to users as tabs at the top of your report. These can be used similarly to sheets in an Excel document.

To add a page, use the `dp.Page` block at the top-level of your report, and give it a title with the `title` parameter.

!!! info 
    
    Pages cannot be nested, and can only exist at the root level of your `dp.Report` object. If you're using pages, all other blocks must be contained inside a Page block.&#x20;

In [13]:
import seaborn as sns
import altair as alt
import datapane as dp

titanic = sns.load_dataset("titanic")

points = (
    alt.Chart(titanic)
    .mark_point()
    .encode(
        x="age:Q",
        color="class:N",
        y="fare:Q",
    )
    .interactive()
    .properties(width="container")
)

report = dp.Report(
    dp.Page(title="Titanic Dataset", blocks=["### Dataset", titanic]),
    dp.Page(title="Titanic Plot", blocks=["### Plot", points]),
)

report.save(path="page.html")

Report saved to ./page.html

In [23]:
from dpdocsutils import previews
previews.embed_local_report('/tutorials/page.html', width="100%", height=740)

## Tabs and Selects

In addition to top-level pages elements, you can include tabs and dropdown selects inside your reports using the `dp.Select` block. This allows users to switch between multiple blocks interactively and can even function as a basic filter. It is also useful for providing background context to another block - for instance, to add source code to a plot or a dataset.

The `dp.Select` block has two modes:&#x20;

* **Tabs** are used for less than 5 options - you can override this default by passing in the parameter `type=dp.SelectType.TABS`
* **Drop downs** are used for 5 or more options - you can override this default by passing in the parameter`type = dp.SelectType.DROPDOWN`. In addition, a search bar will appear if the block contains more than 10 options.&#x20;

To set the option names, make sure each block contained inside your `dp.Select` has a `label` - see example:

In [18]:
import seaborn as sns
import altair as alt
import datapane as dp

code = """
titanic = sns.load_dataset("titanic")

dp.Report(
    dp.Select(blocks=[
        dp.Table(titanic.describe(), label='Data Description'),
        dp.DataTable(titanic, label='Whole Dataset'),
        dp.Code(code, label='Source code')
    ])
).upload(name='altair_example_select')
"""

report = dp.Report(
    "# Titanic overview",
    dp.HTML(
        '<html><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/RMS_Titanic_3.jpg/1599px-RMS_Titanic_3.jpg" style="height:400px;display:flex;margin:auto"></img></html>'
    ),
    dp.Select(
        blocks=[
            dp.Table(titanic.describe(), label="Data Description"),
            dp.DataTable(titanic, label="Whole Dataset"),
            dp.Code(code, label="Source code"),
        ]
    ),
)

report.save(path="select.html")

Report saved to ./select.html

In [21]:
from dpdocsutils import previews
previews.embed_local_report('/tutorials/select.html', width="100%", height=1000)